# Compare agg_rf and seg_glm results
Where does random forests outperfm seg_glm?

In [1]:
ls()

character(0)

In [2]:
options(repr.matrix.max.cols=50, repr.matrix.max.rows=100)

In [3]:
library(dplyr)
library(data.table)
library(partykit)
library(tictoc)
library(caret)
library(e1071)
library(randomForest)
library(ranger)

#for 3d plotting
library(akima)
library(plotly)

# for prep data
library(rPython)
library(stringr)
library(pbapply)
library(stringdist)
library(data.table)

getwd()


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


Loading required package: grid

Loading required package: libcoin

Loading required package: mvtnorm

Loading required package: lattice

Loading required package: ggplot2

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:ggplot2’:

    margin


The following object is masked from ‘package:dplyr’:

    combine



Attaching package: ‘ranger’


The following object is masked from ‘package:randomForest’:

    importance



Attaching package: ‘plotly’


The following object is masked from ‘package:ggplot2’:

    last_plot


The following object is masked from ‘package:sta

[1] "/home/jupyter/local/Domains_202003/scripts/phaseII_04_fusion"

In [4]:

source('../orig/functions.R')

source('../orig/functions_models.R')

source('../phaseII_03_forest/functions_eval.R')


source('../phaseII_03_forest/load_prep_data_expiry.R')

# expiry_train_prepped_2_1 (list, less df's w/ 0 obs)
# expiry_test_prepped_2_1  (list, less df's w/ 0 obs)
# expiry_train_df_1 (above, rbound)
# expiry_test_df_1 (above, rbound)
# expiry_train_df_sub (subset vars)                                        
# expiry_test_df_sub (subset vars)

In [10]:
# processed glm test data
filename = 'seg_glm_expiry_exptest'
system(paste(sprintf("gsutil cp gs://data_input/%s /home/jupyter/local/Domains_202003/data/", filename)))
seg_glm_expiry_exptest <- readRDS(file.path("/home/jupyter/local/Domains_202003/data/", filename))

In [8]:
# processed rf test data
filename = 'predict_ranger_03_expiry2'
system(paste(sprintf("gsutil cp gs://data_output/outputt/%s /home/jupyter/local/Domains_202003/data/", filename)))
load(file.path("/home/jupyter/local/Domains_202003/data/", filename))

# Combine results

In [13]:
dim(seg_glm_expiry_exptest)
head(seg_glm_expiry_exptest)

[1] 296797     40

renewal_type,renewed_count,expiry_date,domain_id,domain,creation_date,status,tld,registrar,reseller,reseller_country,region,reg_period,registrant_country,renewal_status,renew_mbg,renew_type,autorenew_type,renew_date,renew_registrar,renew_reseller,reg_revenue,reg_arpt,renew_period,renew_domain_revenue,renew_arpt,reg_arpt_org,tld_registrar_index,sld,sld_type,sld_length,sld_type2,day_domains,log_reg_arpt,gibb_score,pattern,cluster,pattern_score,pattern_domain_count,first_renewal_prediction
<chr>,<int>,<date>,<int>,<chr>,<date>,<chr>,<fct>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<fct>,<int>,<chr>,<chr>,<date>,<chr>,<chr>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<fct>,<int>,<chr>,<int>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>
FirstTime,1,2020-01-22,91639803,segeln.fun,2019-01-22,Active,fun,1&1 internet,1&1 internet,Germany,Non China,1,NA,Renewed,0,auto-renewal,realized,2020-01-22,1&1 Internet,1&1 Internet,15,15,1,15,15,15,fun1&1 internet,segeln,l,6,6l,1,2.708050,2.23,segeln,1,1,1,0.8459141
FirstTime,1,2020-03-07,95819759,harzhotel.fun,2019-03-07,Active,fun,1&1 internet,1&1 internet,Germany,Non China,1,NA,Renewed,0,auto-renewal,realized,2020-03-07,1&1 Internet,1&1 Internet,15,15,1,15,15,15,fun1&1 internet,harzhotel,l,9,9l,1,2.708050,3.45,harzhotel,1,1,1,0.4607455
FirstTime,1,2020-01-15,91044973,movistar.fun,2019-01-15,Active,fun,10dencehispahard,10dencehispahard,Spain,Non China,1,NA,Not Renewd,0,auto-renewal,unrealized,2020-01-15,10dencehispahard,10dencehispahard,15,15,1,15,15,15,fun10dencehispahard,movistar,l,8,8l,1,2.708050,8.62,movistar,1,1,1,NA
FirstTime,1,2020-02-01,92472913,kuinik.fun,2019-02-01,Active,fun,10dencehispahard,10dencehispahard,Spain,Non China,1,NA,Not Renewd,0,auto-renewal,unrealized,2020-02-01,10dencehispahard,10dencehispahard,15,15,1,15,15,15,fun10dencehispahard,kuinik,l,6,6l,1,2.708050,3.13,kuinik,1,1,1,NA
FirstTime,1,2020-01-21,91532433,bulgari.fun,2019-01-21,Active,fun,1api,1api,Germany,Non China,1,NA,Renewed,0,auto-renewal,realized,2020-01-21,1API,1API,5,5,1,15,15,5,fun1api,bulgari,l,7,7l,1,1.609438,4.51,bulgari,1,1,1,0.9999971
FirstTime,1,2020-04-24,101717674,gotsome.fun,2019-04-24,Active,fun,1api,1api,Germany,Non China,1,NA,Not Renewd,0,auto-renewal,unrealized,2020-04-24,1API,1API,5,5,1,15,15,5,fun1api,gotsome,l,7,7l,1,1.609438,6.09,gotsome,1,1,1,0.9999992


In [14]:
length(predict_ranger_03_expiry2)
head(predict_ranger_03_expiry2)

[1] 296797

[1] 0.4887504 0.4651045 0.5977130 0.5226652 0.4805170 0.4653271

In [15]:
seg_glm_expiry_exptest$rf_predict <- predict_ranger_03_expiry2